<div>
<img src='http://www-scf.usc.edu/~ghasemig/images/sharif.png' alt="SUT logo" width=220 height=220 align=left class="saturate">

<br>
<font face="Times New Roman">
<div dir=ltr align=center> 
<!-- <font color=0F5298 size=7> -->
<font color=0F5298 size=6>
    Introduction to Machine Learning <br> <br>
<!-- <font color=2565AE size=5> -->
<font size=5>
    Computer Engineering Department <br>
    Spring 2023 <br> <br>
<font color=606060 size=5>
    Homework 2: Practical - LDA & QDA <br> <br>
<font color=686880 size=4>
    TAs: Arman Malekzadeh - Arshia Soltani Moakhar
    
____

### Full Name : Mohammad Javad Maheronnaghsh
___

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.metrics import classification_report
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score

# Exploration (20 points)

In [2]:
!wget https://github.com/sharif-ml/sharif-ml.github.io/raw/main/assignments/dataset-train.csv

--2023-04-19 00:01:33--  https://github.com/sharif-ml/sharif-ml.github.io/raw/main/assignments/dataset-train.csv
Resolving github.com (github.com)... 192.30.255.113
Connecting to github.com (github.com)|192.30.255.113|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/sharif-ml/sharif-ml.github.io/main/assignments/dataset-train.csv [following]
--2023-04-19 00:01:33--  https://raw.githubusercontent.com/sharif-ml/sharif-ml.github.io/main/assignments/dataset-train.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 89401 (87K) [text/plain]
Saving to: ‘dataset-train.csv’

dataset-train.csv   100%[===================>]  87.31K  --.-KB/s    in 0.008s  

2023-04-19 00:01:33 (11.0 MB/s) - ‘dataset-train.c

Load the dataset as a dataframe

In [3]:
df = pd.read_csv('dataset-train.csv')

Get the minimum "Area Income" for all Italians

In [4]:
# just Italians
italians = df[df.Country == 'Italy']
# get the min
italians['Area Income'].min()

54286.1

Make a new dataframe called "sample_df" by sampling 100 rows from the previously made dataframe

In [5]:
sample_df = df.sample(n = 100)

Compare all men and women aged less than 30 years in terms of their average "Daily Internet Usage". Your result should contain the average daily usage for each age and gender, seperately.

In [6]:
from numpy.lib.function_base import average
booleans_women = (sample_df.Age < 30) & (sample_df.Gender == 'female')
booleans_men = (sample_df.Age < 30) & (sample_df.Gender == 'male')
women_data = sample_df[booleans_women]
men_data = sample_df[booleans_men]
print("Mean DIU of Women under 30 is " + str(average(women_data['Daily Internet Usage'])))
print("Mean DIU of Men under 30 is " + str(average(men_data['Daily Internet Usage'])))

Mean DIU of Women under 30 is 184.4093333333333
Mean DIU of Men under 30 is 204.565625


# Preprocessing (30 points)

Remove the following columns: "Ad Topic Line", "Timestamp"

In [7]:
sample_df = df # Comment or Uncomment
sample_df = sample_df.drop(['Ad Topic Line', 'Timestamp'], axis = 1)

Convert the "City" and "Country" columns to their one-hot encoded versions.
For each set of one-hot encoded columns, remove one of them to make the columns not correlated. Finally, remove the original columns named "City" and "Country".

In [8]:
cities = pd.Series(sample_df['City'])
dummies = pd.get_dummies(cities)
sample_df = pd.concat([sample_df, dummies], axis = 1)

In [9]:
countries = pd.Series(sample_df['Country'])
dummies = pd.get_dummies(countries)
sample_df = pd.concat([sample_df, dummies], axis = 1)

In [10]:
city_to_delete = cities.head(1)
city_to_delete = str(city_to_delete.tolist()[0])

country_to_Delete = countries.head(1)
country_to_Delete = str(country_to_Delete.tolist()[0])

sample_df = sample_df.drop([city_to_delete, country_to_Delete], axis = 1)

In [11]:
sample_df = sample_df.drop(['City', 'Country'], axis = 1)

Convert "Gender" column to a new binary column (having only 0 and 1 values). Then remove the original column.

In [12]:
genders = pd.Series(sample_df['Gender'])
dummies = pd.get_dummies(genders)
sample_df = pd.concat([sample_df, dummies], axis = 1)

In [13]:
sample_df = sample_df.drop(['Gender', 'female'], axis = 1)

In [14]:
sample_df = sample_df.rename(columns={"male": "Gender"})

By dropping the "Clicked on Ad" column, make a new dataframe called "features_df"

In [15]:
sample_df

,Daily Time Spent on Site,Age,Area Income,Daily Internet Usage,Clicked on Ad,Adamsbury,Alanview,Alexanderfurt,Alexanderview,Alexisland,...,Uzbekistan,Vanuatu,Venezuela,Vietnam,Wallis and Futuna,Western Sahara,Yemen,Zambia,Zimbabwe,Gender
0,68.95,35,61833.90,256.09,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,80.23,31,68441.85,193.77,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,69.47,26,59785.94,236.50,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,74.15,29,54806.18,245.89,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,68.37,35,73889.99,225.58,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
795,75.64,29,69718.19,204.82,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
796,79.22,27,63429.18,198.79,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
797,77.05,34,65756.36,236.08,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
798,66.83,46,77871.75,196.17,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [16]:
features_df = sample_df.drop(['Clicked on Ad'], axis = 1)

Make a new dataframe called "target_df" containing only the "Clicked on Ad" column

In [17]:
target_df = sample_df['Clicked on Ad']

Make two numpy arrays called "x" and "y" from "features_df" and "target_df" dataframes, respectively.

In [18]:
x = features_df.to_numpy()
y = target_df.to_numpy()

# Training Models (40 points)

Split the dataset (`x` and `y`) into "training" and "testing" parts containing 80% and 20% of the data respectively.

In [19]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.20, random_state=42)

Train both LDA and QDA models on the data using `sklearn`

In [20]:
from sklearn import discriminant_analysis
LDA_Sklearn = discriminant_analysis.LinearDiscriminantAnalysis()
LDA_Sklearn.fit(x_train, y_train)
print(LDA_Sklearn)

LinearDiscriminantAnalysis()


In [21]:
QDA_Sklearn = discriminant_analysis.QuadraticDiscriminantAnalysis()
QDA_Sklearn.fit(x_train, y_train)
print(QDA_Sklearn)

/usr/local/lib/python3.9/dist-packages/sklearn/discriminant_analysis.py:926: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")


QuadraticDiscriminantAnalysis()


Train LDA and QDA using your own code!

Linear Discriminant Analysis

In [26]:
class LinearDiscriminantAnalysis:
    """
    Fits it on data, then uses predict to get results.
    """

    def __init__(self):
        self.means = None
        self.covariance = None
        self.priors = None
        self.classes = None

    
    def fit(self, X, y):
        """Fit the training data
        Parameters
        ----------
        X : array-like, shape = [n_samples, n_features]
            Features
        y : array-like, shape = [n_samples, n_labels]
            Labels
        Returns
        -------
        self : object
        """

        n_samples, n_features = X.shape
        # n_samples_2, n_labels = y.shape

        self.classes = np.unique(y)
        n_classes = len(self.classes)
        self.priors = np.zeros(n_classes)
        self.means = np.zeros((n_classes, n_features))
        self.covariances = np.zeros((n_classes, n_features, n_features))
        self.xbar = np.zeros((1, n_features))

        for idx, c in enumerate(self.classes):
            X_c = X[y == c]
            self.priors[idx] = X_c.shape[0] / float(n_samples)
            self.means[idx, :] = X_c.mean(axis=0)
            self.covariances[idx, :, :] = np.cov(X_c.T)
            self.xbar += self.priors[idx] * self.means[idx, :]

        self.covariance = np.zeros((n_features, n_features))
        for idx, c in enumerate(self.classes):
            self.covariance += self.priors[idx] * self.covariances[idx, :, :]

        self.covariance = np.cov(X.T)

        self.coef_ = np.zeros((n_classes, n_features))
        self.intercept_ = np.zeros(n_classes)
        for idx, c in enumerate(self.classes):
            try:
                self.coef_[idx, :] = np.dot(np.linalg.inv(self.covariance), (self.means[idx, :] - self.xbar).T)
                self.intercept_[idx] = -0.5 * np.dot(np.dot(self.means[idx, :], np.linalg.inv(self.covariance)), self.means[idx, :].T) + 0.5 * np.dot(np.dot(self.xbar, np.linalg.inv(self.covariance)), self.xbar.T) + np.log(self.priors[idx])
            except np.linalg.LinAlgError:
                self.coef_[idx, :] = np.dot(np.linalg.pinv(self.covariance), (self.means[idx, :] - self.xbar).T)
                self.intercept_[idx] = -0.5 * np.dot(np.dot(self.means[idx, :], np.linalg.pinv(self.covariance)), self.means[idx, :].T) + 0.5 * np.dot(np.dot(self.xbar, np.linalg.pinv(self.covariance)), self.xbar.T) + np.log(self.priors[idx])

        return self
    
    def predict(self, X):
        """ Predicts the value after the model has been trained.
        Parameters
        ----------
        X : array-like, shape = [n_samples, n_features]
            Features
        Returns
        -------
        Predicted value
        """
        n_samples, n_features = X.shape

        y_pred = np.zeros(n_samples)
        for idx, x in enumerate(X):
            y_pred[idx] = self.classes[np.argmax(np.dot(self.coef_, x) + self.intercept_)]

        return y_pred

        # return self

Quadratic Discriminant Analysis (Optional) (15 bonus points)

In [25]:
class QuadraticDiscriminantAnalysis:
    """
    Fits it on data, then uses predict to get results.
    """

    def __init__(self):
        self.means = None

        self.covariance = None
        self.priors = None
        self.classes = None
        self.xbar = None
        self.coef_ = None
        self.intercept_ = None


        # return self
    
    def fit(self, X, y):
        """Fit the training data
        Parameters
        ----------
        X : array-like, shape = [n_samples, n_features]
            Features
        y : array-like, shape = [n_samples, n_labels]
            Labels
        Returns
        -------
        self : object
        """

        n_samples, n_features = X.shape

        # n_samples_2, n_labels = y.shape

        self.classes = np.unique(y)
        n_classes = len(self.classes)
        self.priors = np.zeros(n_classes)
        self.means = np.zeros((n_classes, n_features))
        self.covariances = np.zeros((n_classes, n_features, n_features))
        self.xbar = np.zeros((1, n_features))

        for idx, c in enumerate(self.classes):
            X_c = X[y == c]
            self.priors[idx] = X_c.shape[0] / float(n_samples)
            self.means[idx, :] = X_c.mean(axis=0)
            self.covariances[idx, :, :] = np.cov(X_c.T)
            self.xbar += self.priors[idx] * self.means[idx, :]

        self.covariance = np.zeros((n_features, n_features))
        for idx, c in enumerate(self.classes):
            self.covariance += self.priors[idx] * self.covariances[idx, :, :]

        self.covariance = np.cov(X.T)

        self.coef_ = np.zeros((n_classes, n_features))
        self.intercept_ = np.zeros(n_classes)
        for idx, c in enumerate(self.classes):
            try:
                self.coef_[idx, :] = np.dot(np.linalg.inv(self.covariance), (self.means[idx, :] - self.xbar).T)
                self.intercept_[idx] = -0.5 * np.dot(np.dot(self.means[idx, :], np.linalg.inv(self.covariance)), self.means[idx, :].T) + 0.5 * np.dot(np.dot(self.xbar, np.linalg.inv(self.covariance)), self.xbar.T) + np.log(self.priors[idx])
            except np.linalg.LinAlgError:
                self.coef_[idx, :] = np.dot(np.linalg.pinv(self.covariance), (self.means[idx, :] - self.xbar).T)
                self.intercept_[idx] = -0.5 * np.dot(np.dot(self.means[idx, :], np.linalg.pinv(self.covariance)), self.means[idx, :].T) + 0.5 * np.dot(np.dot(self.xbar, np.linalg.pinv(self.covariance)), self.xbar.T) + np.log(self.priors[idx])


        return self
    
    def predict(self, X):
        """ Predicts the value after the model has been trained.
        Parameters
        ----------
        X : array-like, shape = [n_samples, n_features]
            Features
        Returns
        -------
        Predicted value
        """
        n_samples, n_features = X.shape

        y_pred = np.zeros(n_samples)
        for idx, x in enumerate(X):
            y_pred[idx] = self.classes[np.argmax(np.dot(self.coef_, x) + self.intercept_)]

        return y_pred


In [27]:
# Train your own models
LDA_Own = LinearDiscriminantAnalysis()
# print(x_train.shape)
# print(y_train.shape)
# print(len(x_train))
# print(len(y_train))
LDA_Own.fit(x_train, y_train)



QDA_Own = quadraticdiscriminantanalysis()
QDA_Own.fit(x_train, y_train)

ValueError: ignored

# Evaluation (10 points)

Using all 4 models, predict the labels for all samples belonging to the "testing" part

In [ ]:
y_pred_sklda = LDA_Sklearn.predict(x_test)
y_pred_skqda = QDA_Sklearn.predict(x_test)


y_pred_lda = LDA_Own.predict(x_test)
y_pred_qda = QDA_Own.predict(x_test)

Calculate Accuracy, Recall, and F1-Scores

In [ ]:
from sklearn import metrics
print(metrics.classification_report(y_test, y_pred_sklda))
print(metrics.classification_report(y_test, y_pred_skqda))

In [ ]:
print(metrics.classification_report(y_test, y_pred_lda))
print(metrics.classification_report(y_test, y_pred_qda))

Compute the ROC AUC Score using `sklearn` (for both LDA and QDA models trained using this library)

In [ ]:
from sklearn.metrics import roc_auc_score
print(roc_auc_score(y_test, y_pred_sklda))

print(roc_auc_score(y_test, y_pred_skqda))

In [ ]:
print(roc_auc_score(y_test, y_pred_lda))

print(roc_auc_score(y_test, y_pred_qda))

# Kaggle Competition

Link: https://www.kaggle.com/t/988b0aa2db2446fd85cff571d69c6386

In [ ]:
# load a local .csv file and train and predict the data
from google.colab import drive 
drive.mount('/content/gdrive')

In [ ]:
df_train = pd.read_csv('/content/gdrive/My Drive/assignment2-train-data.csv')
df_test = pd.read_csv('/content/gdrive/My Drive/assignment2-test-data.csv')
df_train = df_train.drop(['Ad Topic Line', 'Timestamp'], axis = 1)
df_test = df_test.drop(['Ad Topic Line', 'Timestamp'], axis = 1)

In [ ]:
cities1 = pd.Series(df_train['City'])
dummies1 = pd.get_dummies(cities1)
df_train = pd.concat([df_train, dummies1], axis = 1)

cities2 = pd.Series(df_test['City'])
dummies2 = pd.get_dummies(cities2)
df_test = pd.concat([df_test, dummies2], axis = 1)

In [ ]:
countries1 = pd.Series(df_train['Country'])
dummies1 = pd.get_dummies(countries1)
df_train1 = pd.concat([df_train, dummies1], axis = 1)

countries2 = pd.Series(df_test['Country'])
dummies2 = pd.get_dummies(countries2)
df_test = pd.concat([df_test, dummies2], axis = 1)

In [ ]:
city_to_delete1 = cities1.head(1)
city_to_delete1 = str(city_to_delete1.tolist()[0])

country_to_Delete1 = countries1.head(1)
country_to_Delete1 = str(country_to_Delete1.tolist()[0])

df_train = df_train.drop([city_to_delete1, country_to_Delete], axis = 1)

city_to_delete2 = cities2.head(1)
city_to_delete2 = str(city_to_delete2.tolist()[0])

country_to_Delete = countries2.head(1)
country_to_Delete = str(country_to_Delete.tolist()[0])
df_test = df_test.drop([city_to_delete, country_to_Delete], axis = 1)

In [ ]:
df_train = df_train.drop(['City', 'Country'], axis = 1)
df_test = df_test.drop(['City', 'Country'], axis = 1)

In [ ]:
genders = pd.Series(df_train['Gender'])
dummies = pd.get_dummies(genders)
df_train = pd.concat([df_train, dummies], axis = 1)

genders = pd.Series(df_test['Gender'])
dummies = pd.get_dummies(genders)
df_test = pd.concat([df_test, dummies], axis = 1)

In [ ]:
df_train = df_train.drop(['Gender', 'female'], axis = 1)
df_test = df_test.drop(['Gender', 'female'], axis = 1)

In [ ]:
df_train = df_train.rename(columns={"male": "Gender"})
df_test = df_test.rename(columns={"male": "Gender"})

In [ ]:
features_df_train = df_train.drop(['Clicked on Ad'], axis = 1)
features_df_test = df_train

In [ ]:
target_df_train = df_train['Clicked on Ad']

In [ ]:
x = features_df_train.to_numpy()
y = target_df_train.to_numpy()
x_test = features_df_test.to_numpy()

In [ ]:
x_train, y_train = x, y
x_test = x_test

In [ ]:
from sklearn import discriminant_analysis
LDA_Sklearn = discriminant_analysis.LinearDiscriminantAnalysis()
LDA_Sklearn.fit(x_train, y_train)
print(LDA_Sklearn)

QDA_Sklearn = discriminant_analysis.QuadraticDiscriminantAnalysis()
QDA_Sklearn.fit(x_train, y_train)
print(QDA_Sklearn)

In [ ]:
y_pred_sklda = LDA_Sklearn.predict(x_test)
y_pred_skqda = QDA_Sklearn.predict(x_test)